# Pair Programming Métricas

En el pair programming anterior creastéis vuestro primer modelo de machine learning usando la regresion Lineal. Es el momento, que con vuestros datos evaluéis si es bueno haciendo predicciones. Los objetivo de este pairprogramming son:

### 1. Calculéis las métricas para vuestro modelo


### 2. Discutid los resultados de las métricas y extraed conclusiones.


### 3. Guardad los resultados de las métricas en un csv para usarlo más adelante.